# T4 AMP vs FP32 Optimized Research Pipeline

## Research Question
Under what conditions does Automatic Mixed Precision (AMP) actually improve performance on a T4 GPU?

## Performance Optimizations
- Intelligent grid search with adaptive filtering
- Experiment caching to eliminate duplicate work
- Model pooling to reduce initialization overhead
- Parallel execution with efficient resource management
- Statistical early stopping to reduce unnecessary repeats
- Phased approach (coarse → fine search)

## Expected Performance Improvement: 6-10x faster execution

In [ ]:
# Installation and Setup
!pip install -q transformers accelerate datasets pandas seaborn nvidia-ml-py matplotlib tqdm

import os
# Make the allocator resilient and disable cudagraphs globally to avoid overwrite errors.
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True,max_split_size_mb:128")
os.environ.setdefault("CUDA_LAUNCH_BLOCKING", "0")
# Some builds honor this env var for a full cudagraphs kill-switch:
os.environ.setdefault("TORCHINDUCTOR_DISABLE_CUDAGRAPHS", "1")

import torch
import time
import json
import csv
import itertools
import math
import random
import shutil
import zipfile
import threading
import pickle
import multiprocessing
import numpy as np
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from pathlib import Path

import warnings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pynvml
import torch.profiler as prof
from torch import nn
from tqdm import tqdm

from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers.models.glm4.configuration_glm4 import Glm4Config
from datasets import load_dataset

# --- IMPORTANT: Disable Inductor CUDA Graphs to avoid tensor overwrite errors ---
try:
    from torch._inductor import config as inductor_config
    # Older flags:
    inductor_config.triton.cudagraphs = False
    # Newer flag (if present):
    if hasattr(inductor_config, "cuda"):
        inductor_config.cuda.disable_cudagraphs = True
except Exception:
    pass

# (Optional) Quiet a couple of noisy but benign warnings
warnings.filterwarnings("ignore", message="Online softmax is disabled")
warnings.filterwarnings("ignore", message="_maybe_guard_rel")

# Configuration
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

device = 'cuda'
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

RANDOM_SEED = 42
CACHE_DIR = "experiment_cache"
ARTEFACTS_DIR = "artefacts"


## Hyperparameter Configuration

In [ ]:
# --- Configuration (RAM-safe toggles) ---
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

device = 'cuda'
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

RANDOM_SEED = 42
CACHE_DIR = "experiment_cache"
ARTEFACTS_DIR = "artefacts"

# --- Model Configuration ---
LAYERS_LIST     = [4, 8, 12]
D_MODEL_LIST    = [128, 256, 512]
EXPERTS         = 4
TOPK            = 2
VOCAB           = 32000

# --- Data Configuration ---
BS_LIST         = [1, 2, 4, 8, 16]
SEQLEN_LIST     = [64, 128, 256, 512]

# --- Training Configuration ---
PREC_LIST       = ['fp32', 'amp']
REPEATS         = 5
WARMUP          = 3

# --- System Configuration ---
COMPILE_LIST    = [False]
STATIC_LIST     = [False, True]

# --- Optimizer Configuration ---
FUSED_ADAM      = True
FOREACH         = False
LR              = 1e-4
OPTIMIZER_PRIMARY   = "adamw"
OPTIMIZER_FALLBACK  = "sgd"

# --- Profiler & Energy Configuration ---
ENABLE_PROF     = False
POWER_SAMPLE_MS = 50    # interval for polling (still used, but we don't store an array)

# --- Optimization Parameters ---
MAX_WORKERS     = 1
MIN_REPEATS     = 3
CONFIDENCE_THRESHOLD = 0.1
PROFILE_RATE    = 0.05
MAX_PROFILES    = 10

# --- Coarse-to-Fine Search (keep coarse small) ---
INITIAL_LAYERS_LIST     = [4, 12]
INITIAL_D_MODEL_LIST    = [128]
INITIAL_BS_LIST         = [2, 8, 16]
INITIAL_SEQLEN_LIST     = [64, 256]
INITIAL_COMPILE_LIST    = [False]
INITIAL_STATIC_LIST     = [False, True]

# --- NEW: RAM saver switches ---
STORE_TIMES_PER_RUN     = False   # don't keep run timing arrays in result dicts
STORE_ENV_PER_EXPERIMENT= False   # include env info only in PR summary, not every result

print("🔧 Configuration complete!")
initial_coarse_total = len(INITIAL_LAYERS_LIST) * len(INITIAL_D_MODEL_LIST) * len(INITIAL_BS_LIST) * len(INITIAL_SEQLEN_LIST) * len(PREC_LIST) * len(INITIAL_COMPILE_LIST) * len(INITIAL_STATIC_LIST)
print(f"📊 Total experiments in *initial coarse grid*: {initial_coarse_total}")
full_grid_total = len(LAYERS_LIST) * len(D_MODEL_LIST) * len(BS_LIST) * len(SEQLEN_LIST) * len(PREC_LIST) * len(COMPILE_LIST) * len(STATIC_LIST)
print(f"📊 Total experiments in *full potential grid*: {full_grid_total}")


🔧 Configuration complete!
📊 Total experiments in *initial coarse grid*: 48
📊 Total experiments in *full potential grid*: 720


## Core Classes and Utilities

In [ ]:
# --- Core Classes & Utilities (robust pool + improved mem estimators) ---
from collections import deque

class ExperimentCache:
    def __init__(self, cache_dir=CACHE_DIR, window=64):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.ndjson_path = self.cache_dir / "results.ndjson"
        self.keys_path   = self.cache_dir / "completed_keys.txt"
        self.recent = deque(maxlen=window)
        self._completed = set()
        self._load_completed_keys()

    def _load_completed_keys(self):
        if self.keys_path.exists():
            with open(self.keys_path, "r") as f:
                for line in f:
                    k = line.strip()
                    if k:
                        self._completed.add(k)

    def _append_completed_key(self, k: str):
        if k in self._completed: return
        self._completed.add(k)
        with open(self.keys_path, "a") as f:
            f.write(k + "\n")

    def add_result(self, result: dict):
        small = dict(result)
        if not STORE_TIMES_PER_RUN:
            small.pop('times', None)
        if not STORE_ENV_PER_EXPERIMENT:
            small.pop('env', None)
        with open(self.ndjson_path, "a") as f:
            f.write(json.dumps(small) + "\n")
        self.recent.append(small)
        if 'key' in small:
            self._append_completed_key(small['key'])

    def is_completed(self, experiment_key: str) -> bool:
        return experiment_key in self._completed

    def get_result(self, experiment_key: str):
        for r in reversed(self.recent):
            if r.get('key') == experiment_key:
                return r
        if not self.ndjson_path.exists():
            return None
        with open(self.ndjson_path, "r") as f:
            for line in f:
                try:
                    obj = json.loads(line)
                    if obj.get('key') == experiment_key:
                        return obj
                except Exception:
                    pass
        return None

    def read_all(self) -> list:
        out = []
        if not self.ndjson_path.exists():
            return out
        with open(self.ndjson_path, "r") as f:
            for line in f:
                try:
                    out.append(json.loads(line))
                except Exception:
                    pass
        return out

class ModelPool:
    """
    T4-safe model pool:
    - At most one resident model; evict on (layers,d_model,prec) change.
    - Robust to partial failures (avoids KeyError on dict lookups).
    """
    def __init__(self):
        self.models = {}
        self.configs = {}
        self.current_key = None

    def _evict_all(self):
        for k in list(self.models.keys()):
            try:
                del self.models[k]
            except Exception:
                pass
        self.models.clear()
        self.configs.clear()
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()
        torch.cuda.reset_peak_memory_stats()

    def get_model(self, layers, d_model, prec):
        key = f"l{layers}_d{d_model}_p{prec}"
        if self.current_key != key:
            # full eviction before (re)build to reduce fragmentation
            self._evict_all()
            try:
                model, cfg = build_glm4_moe_model(
                    layers, d_model, vocab_size=VOCAB, num_experts=EXPERTS, topk=TOPK
                )
                if (layers * d_model) >= 4096 and hasattr(model, "gradient_checkpointing_enable"):
                    try: model.gradient_checkpointing_enable()
                    except Exception: pass
                self.models[key] = model
                self.configs[key] = cfg
                self.current_key = key
            except Exception as e:
                # ensure clean state on failure, and surface a clear error
                self._evict_all()
                self.current_key = None
                raise RuntimeError(f"ModelPool build failed for {key}: {e}")
        # safe access (avoid KeyError)
        model = self.models.get(key)
        cfg   = self.configs.get(key)
        if model is None or cfg is None:
            # try once more cleanly
            self._evict_all()
            model, cfg = build_glm4_moe_model(
                layers, d_model, vocab_size=VOCAB, num_experts=EXPERTS, topk=TOPK
            )
            self.models[key] = model
            self.configs[key] = cfg
            self.current_key = key
        return model, cfg

    def clear(self):
        self._evict_all()
        self.current_key = None

class SmartProfiler:
    def __init__(self, profile_rate=PROFILE_RATE, max_profiles=MAX_PROFILES):
        self.profile_rate = profile_rate; self.max_profiles = max_profiles; self.profiles_done = 0
    def should_profile(self, layers, d_model, bs, seqlen):
        if self.profiles_done >= self.max_profiles: return False
        complexity_score = (layers * d_model * bs * seqlen) / 1_000_000
        return (0.5 <= complexity_score <= 5.0) and (random.random() < self.profile_rate)
    def increment_profile_count(self): self.profiles_done += 1

def statistical_early_stopping(times, min_repeats=MIN_REPEATS, confidence_threshold=CONFIDENCE_THRESHOLD):
    if len(times) < min_repeats: return False, None
    mean_time = np.mean(times); std_time = np.std(times)
    cv = std_time / mean_time if mean_time > 0 else float('inf')
    return (cv < confidence_threshold, (mean_time if cv < confidence_threshold else None))

def optimize_gpu_memory():
    torch.cuda.empty_cache(); torch.cuda.ipc_collect(); torch.cuda.reset_peak_memory_stats()
    import gc; gc.collect()

# ===== Improved memory estimators (account for attention O(seqlen^2)) =====
def _num_heads_for(d_model: int) -> int:
    return max(8, d_model // 64)

def _estimate_params_moe(layers, d_model, num_experts=EXPERTS, vocab=VOCAB):
    # attention ~ O(d^2), FFN MoE ~ O(E * d^2), very rough
    per_layer = (4 + 8*num_experts) * (d_model**2)
    total = layers * per_layer + vocab * d_model
    return total

def _estimate_param_mem_gb(params, dtype_bytes=4, with_adam=True):
    mult = 3.0 if with_adam else 1.5  # params + grads (+ moments if Adam)
    bytes_total = params * dtype_bytes * mult
    return bytes_total / (1024**3)

def _estimate_activation_mem_gb(layers, d_model, bs, seqlen, prec, topk=TOPK):
    """
    New: include a quadratic attention term.
    - MLP/MoE activations ~ O(layers * bs * seqlen * d * topk)
    - Attention activations ~ O(layers * bs * seqlen^2 * d)   (queries/keys/values & attn maps)
    """
    bytes_per = 2 if prec == 'amp' else 4
    # MLP/MoE (linear-in-seqlen)
    mlp_tokens = layers * bs * seqlen * d_model * max(1, topk)
    # Attention (quadratic-in-seqlen). Use heads*d_head == d_model.
    attn_tokens = layers * bs * (seqlen ** 2) * d_model
    total_tokens = mlp_tokens + 2.0 * attn_tokens   # weight attn a bit more to be conservative
    return (total_tokens * bytes_per) / (1024**3)

def _needs_low_mem_optimizer(layers, d_model):
    return (d_model >= 512) or ((layers * d_model) >= 4096)

# ---- keep the make_opt shim (signature-compatible) ----
def make_opt(*args, **kwargs):
    if len(args) == 1 and isinstance(args[0], nn.Module):
        model = args[0]
        return torch.optim.AdamW(model.parameters(), lr=LR, betas=(0.9, 0.999), eps=1e-8,
                                 weight_decay=0.0, fused=FUSED_ADAM, foreach=FOREACH)
    if len(args) >= 3 and isinstance(args[0], nn.Module):
        model, layers, d_model = args[:3]
        if OPTIMIZER_PRIMARY == "adamw" and not _needs_low_mem_optimizer(layers, d_model):
            return torch.optim.AdamW(model.parameters(), lr=LR, betas=(0.9, 0.999), eps=1e-8,
                                     weight_decay=0.0, fused=FUSED_ADAM, foreach=FOREACH)
        return torch.optim.SGD(model.parameters(), lr=LR)
    raise TypeError("make_opt expects (model) or (model, layers, d_model)")


## Helper Functions

In [ ]:
"""## Helper Functions"""

# NOTE: We define seed_all() here so it's always in scope for run_one_optimized().
# This avoids "name 'seed_all' is not defined" if earlier setup cells weren't rerun.

STRICT_TOKEN_CHECKS = True

def seed_all():
    """Seed all RNGs for reproducibility."""
    try:
        import random
        import numpy as _np
        import torch as _torch
        _SEED = globals().get("RANDOM_SEED", 42)
        random.seed(_SEED)
        _np.random.seed(_SEED)
        _torch.manual_seed(_SEED)
        if _torch.cuda.is_available():
            _torch.cuda.manual_seed_all(_SEED)
    except Exception:
        pass

def get_real_batch(bs: int, seqlen: int, vocab_size: int = None):
    if vocab_size is None:
        vocab_size = VOCAB
    input_ids = torch.randint(
        low=0, high=vocab_size, size=(bs, seqlen),
        device=device, dtype=torch.long
    )
    return input_ids

@torch.no_grad()
def validate_batch(x: torch.Tensor, vocab_size: int):
    if not STRICT_TOKEN_CHECKS:
        return
    if x.dtype != torch.long:
        raise TypeError(f"input_ids dtype must be torch.long, got {x.dtype}")
    if not x.is_cuda:
        raise ValueError("input_ids must be on CUDA")
    x_min = int(x.min().item())
    x_max = int(x.max().item())
    if x_min < 0 or x_max >= vocab_size:
        raise ValueError(
            f"Token id out of range [0, {vocab_size-1}]: min={x_min} max={x_max}"
        )

def make_opt(model: nn.Module):
    return torch.optim.AdamW(
        model.parameters(), lr=LR, betas=(0.9, 0.999), eps=1e-8,
        weight_decay=0.0, fused=FUSED_ADAM, foreach=FOREACH
    )

def safe_clip_grad_norm_(parameters, max_norm):
    """Clip only real, floating-point grads to avoid dtype/nullptr issues."""
    params = [p for p in parameters
              if p.grad is not None and torch.is_floating_point(p.grad)]
    if not params:
        return 0.0
    return torch.nn.utils.clip_grad_norm_(params, max_norm)

def train_step(model: nn.Module, opt: torch.optim.Optimizer,
              scaler: torch.amp.GradScaler | None, input_ids: torch.Tensor,
              prec: str, prof_handler=None) -> float:
    vocab_size = model.config.vocab_size if hasattr(model, 'config') else VOCAB
    validate_batch(input_ids, vocab_size)
    input_ids = input_ids.contiguous()

    # IMPORTANT: use set_to_none=False so grads are real (float) tensors, not None.
    # This avoids "dtype nullptr vs float" in fused/foreach optimizer paths or clipping.
    opt.zero_grad(set_to_none=False)

    use_amp = (prec == 'amp')
    with torch.amp.autocast('cuda', enabled=use_amp, dtype=torch.float16):
        out = model(input_ids=input_ids, labels=input_ids)
        loss = out.loss

    if use_amp:
        scaler.scale(loss).backward()
        scaler.unscale_(opt)
        safe_clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(opt)
        scaler.update()
    else:
        loss.backward()
        safe_clip_grad_norm_(model.parameters(), 1.0)
        opt.step()

    if prof_handler is not None:
        try:
            prof_handler.step()
        except Exception:
            pass
    return float(loss.detach().item())

def train_step_with_grad_accum(model: nn.Module, opt: torch.optim.Optimizer,
                             scaler: torch.amp.GradScaler | None, input_ids: torch.Tensor,
                             prec: str, grad_accum_steps: int = 1, prof_handler=None) -> float:
    """
    Performs a training step with optional gradient accumulation.

    We also mark a cudagraph step boundary (no-op if cudagraphs are disabled)
    to be robust with torch.compile().
    """
    try:
        torch.compiler.cudagraph_mark_step_begin()
    except Exception:
        pass

    if grad_accum_steps == 1:
        return train_step(model, opt, scaler, input_ids, prec, prof_handler)

    micro_batch_size = input_ids.size(0) // grad_accum_steps
    total_loss = 0.0
    for i in range(grad_accum_steps):
        start_idx = i * micro_batch_size
        end_idx = start_idx + micro_batch_size
        micro_input_ids = input_ids[start_idx:end_idx]
        loss = train_step(model, opt, scaler, micro_input_ids, prec, prof_handler)
        total_loss += loss
    return total_loss / grad_accum_steps

def read_power():
    try:
        return pynvml.nvmlDeviceGetPowerUsage(handle) / 1000.0
    except Exception:
        return 0.0

def _nvml_name_to_str(name_obj):
    if isinstance(name_obj, (bytes, bytearray)):
        try:
            return name_obj.decode()
        except Exception:
            return str(name_obj)
    return str(name_obj)

def env_telemetry():
    mem_total_gb = pynvml.nvmlDeviceGetMemoryInfo(handle).total / 1024**3
    raw_name = pynvml.nvmlDeviceGetName(handle)
    name = _nvml_name_to_str(raw_name)
    try:
        cc = pynvml.nvmlDeviceGetCudaComputeCapability(handle)
        cc_str = f"{cc[0]}.{cc[1]}"
    except Exception:
        cc_str = "unknown"
    return {
        "gpu": name,
        "mem_total_gb": mem_total_gb,
        "cc": cc_str,
        "torch": torch.__version__
    }

def cuda_timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    fn()
    end.record()
    torch.cuda.synchronize()
    return start.elapsed_time(end) / 1000.0

def pad_to_multiple(x: torch.Tensor, k: int):
    need = x.size(1) % k
    if need == 0:
        return x
    pad = k - need
    return torch.nn.functional.pad(x, (0, pad), value=0)


## Model Builder

In [ ]:
# ===== Model Builder (unchanged from prior fix, kept here for completeness) =====
from typing import Tuple
import math
import torch
from transformers import AutoModelForCausalLM

try:
    from transformers import Glm4MoeConfig, Glm4MoeForCausalLM
    _HAVE_GLM4_MOE = True
except Exception:
    _HAVE_GLM4_MOE = False

from transformers import Glm4Config, Glm4ForCausalLM

def _compute_intermediate_size(hidden_size: int) -> int:
    return int(math.ceil((8.0 * hidden_size) / 3.0))

def build_glm4_moe_model(
    layers: int,
    d_model: int,
    vocab_size: int = VOCAB,
    num_experts: int = EXPERTS,
    topk: int = TOPK,
    attention_dropout: float = 0.0,
    rms_norm_eps: float = 1e-6,
    device: str = "cuda",
    dtype: torch.dtype = torch.float32,
):
    num_heads = max(8, d_model // 64)
    intermediate_size = _compute_intermediate_size(d_model)

    if _HAVE_GLM4_MOE:
        cfg = Glm4MoeConfig(
            vocab_size=vocab_size,
            hidden_size=d_model,
            num_hidden_layers=layers,
            num_attention_heads=num_heads,
            hidden_act="silu",
            intermediate_size=intermediate_size,
            attention_dropout=attention_dropout,
            rms_norm_eps=rms_norm_eps,
            use_cache=False,
            num_experts=num_experts,
            topk=topk,
            pad_token_id=0,
            eos_token_id=2,
            bos_token_id=1,
        )
        model = Glm4MoeForCausalLM(cfg).to(device=device, dtype=dtype)
    else:
        cfg = Glm4Config(
            vocab_size=vocab_size,
            hidden_size=d_model,
            num_hidden_layers=layers,
            num_attention_heads=num_heads,
            hidden_act="silu",
            intermediate_size=intermediate_size,
            attention_dropout=attention_dropout,
            rms_norm_eps=rms_norm_eps,
            use_cache=False,
            pad_token_id=0,
            eos_token_id=2,
            bos_token_id=1,
        )
        model = Glm4ForCausalLM(cfg).to(device=device, dtype=dtype)

    if (layers * d_model) >= 4096 and hasattr(model, "gradient_checkpointing_enable"):
        try:
            model.gradient_checkpointing_enable()
        except Exception:
            pass

    return model, cfg


## Optimized Experiment Runner

In [ ]:
# --- Optimized Experiment Runner (RAM-safe polling & compact results) ---

def _choose_grad_accum(bs: int, layers: int, d_model: int, seqlen: int, prec: str) -> int:
    if d_model >= 512 or seqlen >= 256:
        return bs  # micro-batch=1
    if bs >= 8 and seqlen >= 128:
        return max(1, bs // 4)
    return 1

def run_one_optimized(layers, d_model, bs, seqlen, prec, compile, static,
                     capacity_factor=1.25, grad_accum_steps=1, lr=1e-4,
                     prof_enabled=False, model_pool=None, profiler=None):
    seed_all()
    optimize_gpu_memory()

    if model_pool is not None:
        model, cfg = model_pool.get_model(layers, d_model, prec=prec)
    else:
        model, cfg = build_glm4_moe_model(layers, d_model)

    if compile:
        print(f"[info] compile requested for ({layers},{d_model},{bs},{seqlen},{prec}) "
              f"but is disabled on T4 for stability.")
    model_to_run = model

    # opt = make_opt(model_to_run, layers, d_model) # Old call with extra args
    opt = make_opt(model_to_run) # Corrected call
    scaler = torch.amp.GradScaler('cuda') if prec == 'amp' else None

    grad_accum_steps = max(grad_accum_steps, _choose_grad_accum(bs, layers, d_model, seqlen, prec))

    input_ids_static = None
    if static:
        static_ids = get_real_batch(bs, seqlen, vocab_size=cfg.vocab_size)
        static_ids = pad_to_multiple(static_ids, 8)
        input_ids_static = static_ids
        validate_batch(input_ids_static, cfg.vocab_size)

    # Warmup
    for _ in range(WARMUP):
        if static:
            input_ids = input_ids_static
        else:
            ids = get_real_batch(bs, seqlen, vocab_size=cfg.vocab_size)
            ids = pad_to_multiple(ids, 8)
            input_ids = ids
        _ = train_step_with_grad_accum(model_to_run, opt, scaler, input_ids, prec, grad_accum_steps)
    torch.cuda.synchronize()

    # ---- RAM-safe power/energy tracking: online (no arrays) ----
    stop_flag = False
    energy_j = 0.0
    avg_power = 0.0
    n_samples = 0
    last_t = time.time()

    def poll_power():
        nonlocal energy_j, avg_power, n_samples, last_t, stop_flag
        while not stop_flag:
            p = read_power()
            now = time.time()
            dt = max(0.0, now - last_t)
            last_t = now
            energy_j += p * dt
            n_samples += 1
            # incremental mean to avoid arrays
            avg_power += (p - avg_power) / n_samples
            time.sleep(POWER_SAMPLE_MS / 1000)

    t = threading.Thread(target=poll_power, daemon=True)
    t.start()

    prof_handler = None
    if prof_enabled and ENABLE_PROF:
        prof_handler = prof.profile(
            activities=[prof.ProfilerActivity.CPU, prof.ProfilerActivity.CUDA],
            record_shapes=True, with_stack=True
        )
        prof_handler.__enter__()

    times = []
    try:
        for r in range(REPEATS):
            if static:
                input_ids = input_ids_static
            else:
                ids = get_real_batch(bs, seqlen, vocab_size=cfg.vocab_size)
                ids = pad_to_multiple(ids, 8)
                input_ids = ids

            sec = cuda_timed(lambda: train_step_with_grad_accum(
                model_to_run, opt, scaler, input_ids, prec, grad_accum_steps,
                (prof_handler if (prof_handler is not None and r == 0) else None)
            ))
            times.append(sec)

            should_stop, _ = statistical_early_stopping(times)
            if should_stop and r >= MIN_REPEATS - 1:
                break

        torch.cuda.synchronize()
    finally:
        if prof_handler is not None:
            prof_handler.__exit__(None, None, None)
            trace_path = f'trace_l{layers}_d{d_model}_bs{bs}_sl{seqlen}_{prec}.json'
            try:
                prof_handler.export_chrome_trace(trace_path)
            except Exception:
                pass
        stop_flag = True
        t.join(timeout=0.2)

    tokens = (input_ids_static.size(0) * input_ids_static.size(1)) if static else (bs * seqlen)
    tps = tokens / (sum(times) / len(times)) if times else 0.0
    mem_gb = torch.cuda.max_memory_allocated() / 1024**3
    env = env_telemetry() if STORE_ENV_PER_EXPERIMENT else None

    # Clean up
    if scaler is not None:
        del scaler
    del opt
    optimize_gpu_memory()

    result = {
        'layers': layers,
        'd_model': d_model,
        'bs': bs,
        'seqlen': seqlen,
        'prec': prec,
        'compile': False,
        'static': static,
        'grad_accum_steps': grad_accum_steps,
        'lr': lr,
        'tps': tps,
        'mem_gb': mem_gb,
        'energy_j': energy_j,
        'avg_power_w': avg_power,
        'repeats_used': len(times)
    }
    if STORE_TIMES_PER_RUN:
        result['times'] = times
    if STORE_ENV_PER_EXPERIMENT:
        result['env'] = env
    return result

## Parallel Experiment Execution

In [ ]:
# --- Parallel Experiment Execution (stronger OOM guard) ---

def _oom_guard(layers, d_model, bs, seqlen, prec, using_adam=True):
    # param footprint (with optimizer state)
    params = _estimate_params_moe(layers, d_model, EXPERTS, VOCAB)
    param_gb = _estimate_param_mem_gb(params, dtype_bytes=4, with_adam=using_adam)
    # activation footprint (includes attention seqlen^2 term)
    act_gb = _estimate_activation_mem_gb(layers, d_model, bs, seqlen, prec)
    # be conservative: params + 2.5x activations + 1GB safety buffer
    total_gb = param_gb + 2.5 * act_gb + 1.0
    # tighter threshold so we skip borderline configs before they fragment VRAM
    return (total_gb > 12.0), total_gb

def run_one_cached(*args, cache=None, model_pool=None, profiler=None):
    layers, d_model, bs, seqlen, prec, compile, static = args
    experiment_key = f"l{layers}_d{d_model}_bs{bs}_sl{seqlen}_{prec}_c{compile}_s{static}"

    using_adam = (OPTIMIZER_PRIMARY == "adamw" and not _needs_low_mem_optimizer(layers, d_model))
    should_skip, est_total = _oom_guard(layers, d_model, bs, seqlen, prec, using_adam=using_adam)
    if should_skip:
        print(f"Skipping by memory guard (~{est_total:.1f} GB): {args}")
        cache.add_result({
            'layers': layers, 'd_model': d_model, 'bs': bs, 'seqlen': seqlen,
            'prec': prec, 'compile': compile, 'static': static,
            'tps': float('nan'), 'mem_gb': float('nan'),
            'energy_j': float('nan'), 'avg_power_w': float('nan'),
            'repeats_used': 0,
            'error': f"Skipped by memory guard (~{est_total:.1f} GB)",
            'key': experiment_key
        })
        return {'key': experiment_key}

    if cache.is_completed(experiment_key):
        cached = cache.get_result(experiment_key)
        return cached if cached else {'key': experiment_key}

    prof_enabled = False
    if profiler is not None:
        prof_enabled = profiler.should_profile(layers, d_model, bs, seqlen)

    try:
        result = run_one_optimized(*args, prof_enabled=prof_enabled,
                                   model_pool=model_pool, profiler=profiler)
    except RuntimeError as e:
        # catch pool/build errors & CUDA OOM and record as a result line (not crash the run)
        cache.add_result({
            'layers': layers, 'd_model': d_model, 'bs': bs, 'seqlen': seqlen,
            'prec': prec, 'compile': compile, 'static': static,
            'tps': float('nan'), 'mem_gb': float('nan'),
            'energy_j': float('nan'), 'avg_power_w': float('nan'),
            'repeats_used': 0,
            'error': str(e),
            'key': experiment_key
        })
        return {'key': experiment_key}

    result['key'] = experiment_key
    cache.add_result(result)
    return result

def parallel_experiment_runner(grid, max_workers=None):
    max_workers = 1
    cache = ExperimentCache()
    model_pool = ModelPool()

    pending = []
    for args in grid:
        layers, d_model, bs, seqlen, prec, compile, static = args
        key = f"l{layers}_d{d_model}_bs{bs}_sl{seqlen}_{prec}_c{compile}_s{static}"
        if not cache.is_completed(key):
            pending.append(args)

    print(f"Total experiments: {len(grid)}")
    print(f"Already completed: {len(grid) - len(pending)}")
    print(f"Pending: {len(pending)}")

    if not pending:
        print("All experiments already completed!")
        return cache.read_all()

    profiler = SmartProfiler()
    results_count = 0

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_args = {
            executor.submit(run_one_cached, *args, cache=cache,
                            model_pool=model_pool, profiler=profiler): args
            for args in pending
        }
        with tqdm(total=len(future_to_args), desc="Running experiments", leave=True) as pbar:
            for future in as_completed(future_to_args):
                try:
                    _ = future.result()
                    results_count += 1
                    pbar.set_postfix({'completed': results_count, 'profiles': profiler.profiles_done})
                except Exception as e:
                    args_failed = future_to_args[future]
                    print(f"Experiment failed: {args_failed}, Error: {e}")
                    layers, d_model, bs, seqlen, prec, compile, static = args_failed
                    cache.add_result({
                        'layers': layers, 'd_model': d_model, 'bs': bs, 'seqlen': seqlen,
                        'prec': prec, 'compile': compile, 'static': static,
                        'tps': float('nan'), 'mem_gb': float('nan'),
                        'energy_j': float('nan'), 'avg_power_w': float('nan'),
                        'repeats_used': 0,
                        'error': str(e),
                        'key': f"l{layers}_d{d_model}_bs{bs}_sl{seqlen}_{prec}_c{compile}_s{static}"
                    })
                pbar.update(1)
                optimize_gpu_memory()

    model_pool.clear()
    return cache.read_all()


## Intelligent Grid Generation

In [ ]:
# --- Intelligent Grid Generation (modified/new filtering) ---
def create_phased_grids():
    # Phase 1: Coarse (kept light on purpose: no d=512)
    layers_coarse = INITIAL_LAYERS_LIST
    d_model_coarse = INITIAL_D_MODEL_LIST
    bs_coarse = INITIAL_BS_LIST
    seqlen_coarse = INITIAL_SEQLEN_LIST

    grid_coarse = list(itertools.product(
        layers_coarse, d_model_coarse, bs_coarse, seqlen_coarse,
        PREC_LIST, INITIAL_COMPILE_LIST, INITIAL_STATIC_LIST
    ))

    # Phase 2: Fine — include 512 but only with safer shapes by default
    layers_fine = [6, 8, 10, 4, 12]
    d_model_fine = [256, 384, 512]
    bs_fine = [2, 4, 6, 8]
    seqlen_fine = [64, 128, 192, 256]

    grid_fine = list(itertools.product(
        layers_fine, d_model_fine, bs_fine, seqlen_fine,
        PREC_LIST, COMPILE_LIST, STATIC_LIST
    ))
    return grid_coarse, grid_fine

def filter_grid_intelligently(base_grid, results_so_far=None):
    if results_so_far is None or len(results_so_far) == 0:
        print(f"🎯 Grid filtered: {len(base_grid)} → {len(base_grid)} (no prior results)")
        return base_grid

    def should_include_config(layers, d_model, bs, seqlen, results):
        if layers >= 12 and bs >= 8 and seqlen >= 512:
            return False
        if layers >= 8 and bs >= 16 and seqlen >= 256:
            return False
        similar = [r for r in results
                   if r.get('layers') == layers and r.get('d_model') == d_model
                   and not np.isnan(r.get('tps', float('nan')))]
        if similar:
            avg_tps = np.mean([r['tps'] for r in similar])
            if avg_tps < 50:
                return False
        return True

    filtered = []
    for (layers, d_model, bs, seqlen, prec, compile, static) in base_grid:
        if should_include_config(layers, d_model, bs, seqlen, results_so_far):
            filtered.append((layers, d_model, bs, seqlen, prec, compile, static))
    print(f"🎯 Grid filtered: {len(base_grid)} → {len(filtered)} experiments")
    return filtered

def analyze_promising_regions(results):
    if not results:
        return {}
    df = pd.DataFrame(results)
    df = df[~df['tps'].isna()]
    if len(df) == 0:
        return {}
    model_performance = df.groupby(['layers', 'd_model'])['tps'].mean().reset_index()
    model_performance = model_performance.sort_values('tps', ascending=False)
    top_models = model_performance.head(3)
    promising_regions = {
        'promising_layers': top_models['layers'].tolist(),
        'promising_d_models': top_models['d_model'].tolist(),
        'best_tps': top_models['tps'].max()
    }
    print(f"Promising regions identified: {promising_regions}")
    return promising_regions

def filter_grid_by_promising_regions(grid, promising_regions):
    if not promising_regions:
        return grid
    filtered_grid = []
    for args in grid:
        layers, d_model, bs, seqlen, prec, compile, static = args
        if (layers in promising_regions.get('promising_layers', []) or
            d_model in promising_regions.get('promising_d_models', [])):
            filtered_grid.append(args)
    print(f"Fine grid filtered: {len(grid)} → {len(filtered_grid)} experiments")
    return filtered_grid

print("🎯 Intelligent grid generation complete!")


🎯 Intelligent grid generation complete!


## Main Execution Pipeline

In [ ]:
# --- Main Execution Pipeline (modified to use filter_grid_intelligently on coarse) ---
def optimized_evaluation_pipeline(use_phased_approach=True):
    print("Starting optimized evaluation pipeline")
    print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    cache = ExperimentCache()

    if use_phased_approach:
        print("Using phased approach (coarse → fine)")

        print("\nPhase 1: Coarse grid search")
        grid_coarse, grid_fine = create_phased_grids()

        # Read existing results from the cache file
        existing_results = cache.read_all()
        # IMPORTANT: filter the provided coarse grid, NOT the full grid
        grid_coarse_filtered = filter_grid_intelligently(grid_coarse, existing_results)

        results_coarse = parallel_experiment_runner(grid_coarse_filtered)

        promising_regions = analyze_promising_regions(results_coarse)

        print("\nPhase 2: Fine grid search")
        grid_fine_filtered = filter_grid_by_promising_regions(grid_fine, promising_regions)

        results_fine = parallel_experiment_runner(grid_fine_filtered)

        all_results = results_coarse + results_fine

    else:
        print("Using single-pass approach")
        # If you want a single full pass, just run the full grid directly:
        full_grid = list(itertools.product(
            LAYERS_LIST, D_MODEL_LIST, BS_LIST, SEQLEN_LIST,
            PREC_LIST, COMPILE_LIST, STATIC_LIST
        ))
        # Read existing results from the cache file
        existing_results = cache.read_all()
        full_grid_filtered = filter_grid_intelligently(full_grid, existing_results)
        all_results = parallel_experiment_runner(full_grid_filtered)

    print(f"Pipeline completed at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Total experiments completed: {len(all_results)}")
    return all_results

## Analysis and Visualization

In [ ]:
# --- Analysis and Visualization (reads from cache.read_all()) ---

def analyze_results(results):
    df = pd.DataFrame(results)
    if 'tps' not in df or df['tps'].isna().all():
        print("No valid results to analyze")
        return None, None

    df = df[~df['tps'].isna()]

    def ci95(x):
        return 1.96 * pd.Series(x).std() / math.sqrt(len(x)) if len(x) > 1 else 0

    stats = (df.groupby(['layers','d_model','bs','seqlen','prec','compile','static'])
           .agg(tps_mean=('tps','mean'),
                tps_ci=('tps',ci95),
                mem_mean=('mem_gb','mean'),
                energy_mean=('energy_j','mean'),
                power_mean=('avg_power_w','mean'),
                repeats_mean=('repeats_used','mean'))
           .reset_index())

    summary = stats.pivot_table(
        index=['layers','d_model','bs','seqlen','compile','static'],
        columns='prec',
        values=['tps_mean','tps_ci','mem_mean','energy_mean']
    )

    if ('tps_mean', 'amp') in summary.columns and ('tps_mean', 'fp32') in summary.columns:
        summary['speedup'] = summary[('tps_mean','amp')] / summary[('tps_mean','fp32')]

    if ('energy_mean', 'amp') in summary.columns and ('energy_mean', 'fp32') in summary.columns:
        summary['energy_ratio'] = summary[('energy_mean','amp')] / summary[('energy_mean','fp32')]

    return df, summary


## Save Results and Artefacts

In [ ]:
def save_artefacts(df, summary):
    os.makedirs(ARTEFACTS_DIR, exist_ok=True)

    df.to_csv(f'{ARTEFACTS_DIR}/full_data.csv', index=False)
    print(f"Raw data saved to {ARTEFACTS_DIR}/full_data.csv")

    summary.to_csv(f'{ARTEFACTS_DIR}/summary.csv')
    print(f"Summary saved to {ARTEFACTS_DIR}/summary.csv")

    env_info = env_telemetry()
    pr_summary = {
        'experiment_info': {
            'timestamp': datetime.now().isoformat(),
            'total_experiments': len(df),
            'successful_experiments': len(df[~df['tps'].isna()]),
            'configuration': {
                'layers_list': LAYERS_LIST,
                'd_model_list': D_MODEL_LIST,
                'bs_list': BS_LIST,
                'seqlen_list': SEQLEN_LIST,
                'prec_list': PREC_LIST,
                'compile_list': COMPILE_LIST,
                'static_list': STATIC_LIST
            }
        },
        'environment': env_info,
        'key_findings': {
            'avg_speedup': summary['speedup'].mean() if 'speedup' in summary.columns else None,
            'avg_energy_ratio': summary['energy_ratio'].mean() if 'energy_ratio' in summary.columns else None,
            'best_configuration': summary.loc[summary['speedup'].idxmax()] if 'speedup' in summary.columns else None
        }
    }

    with open(f'{ARTEFACTS_DIR}/pr_summary.json', 'w') as f:
        json.dump(pr_summary, f, indent=2, default=str)

    print(f"PR summary saved to {ARTEFACTS_DIR}/pr_summary.json")

    with open(f'{ARTEFACTS_DIR}/LICENSE', 'w') as f:
        f.write('Apache-2.0')

    zip_path = f'{ARTEFACTS_DIR}.zip'
    shutil.make_archive(ARTEFACTS_DIR, 'zip', ARTEFACTS_DIR)

    print(f"Artefacts bundle created: {zip_path}")
    print(f"Contains: {len(df)} experiments, {len(summary)} configurations")

    return zip_path

## Execute Complete Pipeline

In [ ]:
print("Starting complete optimized pipeline execution")
print("=" * 60)

try:
    results = optimized_evaluation_pipeline(use_phased_approach=True)

    print("\nAnalyzing results...")
    df, summary = analyze_results(results)

    print("\nCreating visualizations...")
    create_visualizations(df, summary)

    print("\nSaving artefacts...")
    zip_path = save_artefacts(df, summary)

    print("\n" + "=" * 60)
    print("Pipeline execution completed successfully!")
    print(f"Results available in: {zip_path}")

    if summary is not None and 'speedup' in summary.columns:
        print("\nKey Findings:")
        print(f"   • Average AMP/FP32 speedup: {summary['speedup'].mean():.2f}x")
        print(f"   • Best speedup achieved: {summary['speedup'].max():.2f}x")
        print(f"   • Worst speedup observed: {summary['speedup'].min():.2f}x")

        if 'energy_ratio' in summary.columns:
            print(f"   • Average energy ratio (AMP/FP32): {summary['energy_ratio'].mean():.2f}")

except Exception as e:
    print(f"Pipeline execution failed: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# =========================
# Research-Grade Postprocessing
# =========================
import os, json, math, shutil, datetime, textwrap, statistics, itertools
from pathlib import Path
from math import comb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Respect prior config if present
ARTEFACTS_DIR = os.getenv("ARTEFACTS_DIR", "artefacts")
CACHE_DIR     = os.getenv("CACHE_DIR", "experiment_cache")
NDJSON_PATH   = Path(CACHE_DIR) / "results.ndjson"
os.makedirs(ARTEFACTS_DIR, exist_ok=True)

# ---------- helpers ----------
def _safe_read_ndjson(path: Path) -> pd.DataFrame:
    rows = []
    if not path.exists():
        print(f"[warn] NDJSON not found at {path}")
        return pd.DataFrame()
    with open(path, "r") as f:
        for line in f:
            line=line.strip()
            if not line: continue
            try: rows.append(json.loads(line))
            except Exception: pass
    return pd.DataFrame(rows)

def _to_num(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def _ci95(series):
    x = pd.Series(series).dropna().values
    n = len(x)
    if n <= 1: return 0.0
    return 1.96 * (np.std(x, ddof=1) / math.sqrt(n))

def _bootstrap_ci_mean(vals, B=2000, alpha=0.05, seed=42):
    rng = np.random.default_rng(seed)
    vals = np.asarray([v for v in vals if np.isfinite(v)])
    if len(vals) == 0: return (np.nan, np.nan)
    boots = []
    n = len(vals)
    for _ in range(B):
        sample = vals[rng.integers(0, n, size=n)]
        boots.append(np.mean(sample))
    boots = np.sort(boots)
    lo = np.percentile(boots, 100*alpha/2)
    hi = np.percentile(boots, 100*(1-alpha/2))
    return (float(lo), float(hi))

def _binomial_p_two_sided(k, n, p=0.5):
    # exact two-sided binomial (small n safe)
    # sum of probabilities of outcomes as or more extreme than observed
    def pmf(k, n, p):
        return comb(n, k) * (p**k) * ((1-p)**(n-k))
    # central value
    obs = pmf(k, n, p)
    tot = 0.0
    for i in range(0, n+1):
        if pmf(i, n, p) <= obs + 1e-18:
            tot += pmf(i, n, p)
    return min(1.0, 2.0*min(sum(pmf(i,n,p) for i in range(0,k+1)),
                             sum(pmf(i,n,p) for i in range(k,n+1))))

def _winsorize(series, p=0.01):
    x = pd.Series(series).astype(float)
    lo, hi = x.quantile(p), x.quantile(1-p)
    return x.clip(lo, hi)

def _fmt(x, n=3):
    try: return f"{float(x):.{n}g}"
    except: return str(x)

def _maybe_env():
    try:
        return env_telemetry()  # defined earlier in your notebook
    except Exception:
        return {}

# ---------- 1) Load + clean ----------
raw = _safe_read_ndjson(NDJSON_PATH)
if raw.empty:
    raise RuntimeError("No results.ndjson found or it was empty. Run experiments first.")

num_cols = ["layers","d_model","bs","seqlen","tps","mem_gb","energy_j","avg_power_w","repeats_used"]
df = _to_num(raw.copy(), num_cols)
for c in ['compile','static','prec']:
    if c not in df.columns:
        df[c] = False if c != 'prec' else np.nan

# Minimal valid rows
df = df[df["tps"].notna() & df["layers"].notna() & df["d_model"].notna() &
        df["bs"].notna() & df["seqlen"].notna() & df["prec"].notna()].copy()

# ---------- 2) Matched AMP↔FP32 pairs ----------
pair_keys = ['layers','d_model','bs','seqlen','compile','static']
pivot = (df
         .groupby(pair_keys + ['prec'])[['tps','energy_j']] # Corrected: ['tps','energy_j'] is now a list
         .mean().unstack('prec'))
pivot.columns = [f"{m}__{p}" for m,p in pivot.columns]
pivot = pivot.reset_index()

# keep only pairs where both exist
need = ['tps__amp','tps__fp32']
pivot = pivot[[*pair_keys, *[c for c in pivot.columns if c in need or c.startswith("energy_j__")]]].dropna(subset=need)

# Metrics: ratios/percent diffs; use log for stability
pivot['speedup'] = pivot['tps__amp'] / pivot['tps__fp32']
pivot['log_speedup'] = np.log(pivot['speedup'])
if 'energy_j__amp' in pivot.columns and 'energy_j__fp32' in pivot.columns:
    pivot['energy_ratio'] = pivot['energy_j__amp'] / pivot['energy_j__fp32']
else:
    pivot['energy_ratio'] = np.nan

# Winsorize extreme ratios for analysis plots
pivot['speedup_w'] = _winsorize(pivot['speedup'])
pivot['log_speedup_w'] = np.log(pivot['speedup_w'])

# ---------- 3) Descriptive stats ----------
overall_avg_speed = float(np.mean(pivot['speedup']))
overall_med_speed = float(np.median(pivot['speedup']))
mean_log = float(np.mean(pivot['log_speedup']))
ci_lo, ci_hi = _bootstrap_ci_mean(pivot['log_speedup'], B=2000, alpha=0.05)

# Convert log-CI to multiplicative CI
mul_lo, mul_hi = math.exp(ci_lo), math.exp(ci_hi)

# Sign test: count configs where AMP > FP32
wins = int(np.sum(pivot['speedup'] > 1.0))
loss = int(np.sum(pivot['speedup'] < 1.0))
ties = int(np.sum(np.isclose(pivot['speedup'], 1.0, atol=1e-3)))
n_eff = wins + loss
p_two = _binomial_p_two_sided(wins, n_eff, p=0.5) if n_eff > 0 else np.nan

# ---------- 4) Simple regression on log-speedup ----------
# features: intercept, layers, d_model, log(bs), log(seqlen), static (0/1)
X_cols = []
X = []
for _, r in pivot.iterrows():
    X_cols = ['intercept','layers','d_model','log_bs','log_seqlen','static']
    X.append([
        1.0,
        float(r['layers']),
        float(r['d_model']),
        math.log(max(1.0, float(r['bs']))),
        math.log(max(1.0, float(r['seqlen']))),
        1.0 if bool(r['static']) else 0.0
    ])
X = np.asarray(X)
y = pivot['log_speedup'].values
# OLS via lstsq (no regularization)
coefs, *_ = np.linalg.lstsq(X, y, rcond=None)
reg = pd.DataFrame({'feature': X_cols, 'coef': coefs})
# crude standard errors via classical formula
y_hat = X @ coefs
resid = y - y_hat
dof = max(1, len(y) - len(coefs))
sigma2 = float(np.sum(resid**2) / dof)
cov = sigma2 * np.linalg.inv(X.T @ X)
se = np.sqrt(np.diag(cov))
reg['stderr'] = se
reg['t'] = reg['coef'] / reg['stderr']
reg['exp_coef'] = np.exp(reg['coef'])  # multiplicative effect on speedup
reg_path = Path(ARTEFACTS_DIR) / "regression_log_speedup.csv"
reg.to_csv(reg_path, index=False)

# ---------- 5) Stratified tables ----------
# bins
size_proxy = pivot['layers'] * pivot['d_model']
pivot['size_bin'] = pd.qcut(size_proxy, q=min(5, max(2, size_proxy.nunique())), duplicates='drop')
pivot['bs_bin'] = pd.cut(pivot['bs'], bins=sorted(set([1,2,4,6,8,12,16,32])), right=True, include_lowest=True)
pivot['sl_bin'] = pd.cut(pivot['seqlen'], bins=sorted(set([64,96,128,160,192,224,256,320,384,512])), right=True, include_lowest=True)

by_bins = (pivot.groupby(['size_bin','bs_bin','sl_bin'])
                 .agg(n=('speedup','size'),
                      speedup_mean=('speedup','mean'),
                      speedup_med=('speedup','median'),
                      log_speedup_mean=('log_speedup','mean'))
                 .reset_index())
by_bins_path = Path(ARTEFACTS_DIR) / "stratified_bins.csv"
by_bins.to_csv(by_bins_path, index=False)

# ---------- 6) Per-precision summaries (sanity) ----------
per_prec = (df.groupby(['layers','d_model','bs','seqlen','prec'])['tps']
              .agg(['count','mean','median']).reset_index())
per_prec_path = Path(ARTEFACTS_DIR) / "per_precision_stats.csv"
per_prec.to_csv(per_prec_path, index=False)

# ---------- 7) Plots ----------
ts = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Heatmap: speedup by (model) x (workload)
try:
    heat = pivot.copy()
    heat['model'] = heat['layers'].astype(int).astype(str) + "L×" + heat['d_model'].astype(int).astype(str)
    heat['workload'] = "bs" + heat['bs'].astype(int).astype(str) + "_sl" + heat['seqlen'].astype(int).astype(str)
    mat = heat.pivot_table(index='model', columns='workload', values='speedup', aggfunc='mean')
    plt.figure(figsize=(12, max(4, 0.5*len(mat))))
    im = plt.imshow(mat, aspect='auto')
    plt.colorbar(im, label='AMP / FP32 speedup (↑ better)')
    plt.title('AMP vs FP32 Speedup by Model & Workload')
    plt.yticks(range(len(mat.index)), mat.index)
    plt.xticks(range(len(mat.columns)), mat.columns, rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f"{ARTEFACTS_DIR}/speedup_heatmap_{ts}.png", dpi=150, bbox_inches="tight")
    plt.close()
except Exception as e:
    print(f"[warn] heatmap failed: {e}")

# Scatter: speedup vs seqlen / bs
try:
    plt.figure(figsize=(10,6))
    plt.scatter(pivot['seqlen'], pivot['speedup_w'], alpha=0.6, s=32)
    plt.axhline(1.0, color='r', linestyle='--')
    plt.xlabel("Sequence length")
    plt.ylabel("AMP/FP32 speedup (winsorized)")
    plt.title("Speedup vs Sequence Length")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{ARTEFACTS_DIR}/speedup_vs_seqlen_{ts}.png", dpi=150, bbox_inches="tight")
    plt.close()

    plt.figure(figsize=(10,6))
    plt.scatter(pivot['bs'], pivot['speedup_w'], alpha=0.6, s=32)
    plt.axhline(1.0, color='r', linestyle='--')
    plt.xlabel("Batch size")
    plt.ylabel("AMP/FP32 speedup (winsorized)")
    plt.title("Speedup vs Batch Size")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{ARTEFACTS_DIR}/speedup_vs_bs_{ts}.png", dpi=150, bbox_inches="tight")
    plt.close()
except Exception as e:
    print(f"[warn] scatter plots failed: {e}")

# Top tables like before
top_tps = (df.sort_values('tps', ascending=False)
             .head(20)
             [['layers','d_model','bs','seqlen','prec','tps','mem_gb','avg_power_w']])
top_speedup = (pivot.sort_values('speedup', ascending=False)
                    .head(20)
                    [['layers','d_model','bs','seqlen','compile','static','speedup']])
top_tps.to_csv(Path(ARTEFACTS_DIR)/"top20_tps.csv", index=False)
top_speedup.to_csv(Path(ARTEFACTS_DIR)/"top20_speedup.csv", index=False)

# ---------- 8) Save tidy datasets ----------
# (a) raw row-level data
df_csv = Path(ARTEFACTS_DIR)/"full_data.csv"
df.to_csv(df_csv, index=False)
try: df.to_parquet(Path(ARTEFACTS_DIR)/"full_data.parquet", index=False)
except Exception: pass

# (b) matched pairs
pairs_csv = Path(ARTEFACTS_DIR)/"matched_pairs.csv"
pivot.to_csv(pairs_csv, index=False)

# (c) compact summary
stats = (df.groupby(['layers','d_model','bs','seqlen','prec','compile','static'])
           .agg(tps_mean=('tps','mean'),
                tps_ci=('tps', _ci95),
                mem_mean=('mem_gb','mean'),
                energy_mean=('energy_j','mean'),
                power_mean=('avg_power_w','mean'),
                repeats_mean=('repeats_used','mean'))
           .reset_index())
summary = stats.pivot_table(
    index=['layers','d_model','bs','seqlen','compile','static'],
    columns='prec',
    values=['tps_mean','tps_ci','mem_mean','energy_mean'],
    aggfunc='first'
)
if ('tps_mean','amp') in summary.columns and ('tps_mean','fp32') in summary.columns:
    summary['speedup'] = summary[('tps_mean','amp')] / summary[('tps_mean','fp32')]
if ('energy_mean','amp') in summary.columns and ('energy_mean','fp32') in summary.columns:
    summary['energy_ratio'] = summary[('energy_mean','amp')] / summary[('energy_mean','fp32')]
summary_path = Path(ARTEFACTS_DIR)/"summary_by_config.csv"
# flatten for CSV
flat_sum = summary.copy()
flat_sum.columns = ['__'.join(c).strip('_') if isinstance(c, tuple) else c for c in flat_sum.columns]
flat_sum.reset_index().to_csv(summary_path, index=False)

# ---------- 9) Key metrics JSON ----------
env_info = _maybe_env()
key = {
    "generated_at": datetime.datetime.now().isoformat(timespec='seconds'),
    "environment": env_info,
    "counts": {
        "rows": int(len(df)),
        "pairs": int(len(pivot)),
        "unique_settings": int(df[['layers','d_model','bs','seqlen','prec']].drop_duplicates().shape[0])
    },
    "speedup": {
        "avg_ratio": overall_avg_speed,
        "median_ratio": overall_med_speed,
        "log_mean": mean_log,
        "log_mean_ci_95": [ci_lo, ci_hi],
        "mult_mean_ci_95": [mul_lo, mul_hi]
    },
    "sign_test": {
        "wins": wins, "losses": loss, "ties": ties, "two_sided_p": float(p_two)
    },
    "regression_log_speedup": reg.to_dict(orient="records")
}
with open(Path(ARTEFACTS_DIR)/"key_metrics.json","w") as f:
    json.dump(key, f, indent=2)

# ---------- 10) Research-ready REPORT.md ----------
report = Path(ARTEFACTS_DIR)/"REPORT.md"
with open(report, "w") as f:
    f.write("# T4 AMP vs FP32 — Research Report\n\n")
    f.write(f"_Generated: {datetime.datetime.now().isoformat(timespec='seconds')}_\n\n")

    if env_info:
        f.write("## Environment\n")
        f.write(f"- GPU: {env_info.get('gpu','?')}\n")
        f.write(f"- Memory: {_fmt(env_info.get('mem_total_gb','?'))} GB\n")
        f.write(f"- Compute Capability: {env_info.get('cc','?')}\n")
        f.write(f"- PyTorch: {env_info.get('torch','?')}\n\n")

    f.write("## Dataset Stats\n")
    f.write(f"- Total rows: {len(df)}\n")
    f.write(f"- Matched AMP↔FP32 pairs: {len(pivot)}\n")
    uniq = df[['layers','d_model','bs','seqlen','prec']].drop_duplicates().shape[0]
    f.write(f"- Unique (layers,d_model,bs,seqlen,prec) combos: {uniq}\n\n")

    f.write("## Executive Summary\n")
    f.write(f"- Mean AMP/FP32 speedup across **matched** configs: **{_fmt(overall_avg_speed,3)}×** "
            f"(median **{_fmt(overall_med_speed,3)}×**).\n")
    f.write(f"- Mean log-speedup 95% bootstrap CI: **[{_fmt(mul_lo,3)}, {_fmt(mul_hi,3)}]×**.\n")
    if not math.isnan(p_two):
        f.write(f"- Sign test vs parity (AMP=FP32): wins={wins}, losses={loss}, ties={ties}, "
                f"two-sided p≈**{_fmt(p_two,3)}**.\n")
    f.write("\n")

    f.write("## When AMP Helps (Patterns)\n")
    # quick rules of thumb from regression (exp_coef > 1 means positive on speedup)
    f.write("Interpreting coefficients on **log-speedup** (multiplicative effects):\n\n")
    f.write(reg.rename(columns={'exp_coef':'mult_effect'})
            [['feature','coef','stderr','t','mult_effect']]
            .to_markdown(index=False))
    f.write("\n\n")

    f.write("## Top Throughput (absolute TPS)\n")
    f.write(top_tps.to_markdown(index=False))
    f.write("\n\n")

    f.write("## Top AMP Speedups (AMP/FP32)\n")
    f.write(top_speedup.rename(columns={'speedup':'AMP/FP32'}).to_markdown(index=False))
    f.write("\n\n")

    f.write("## Stratified Effects (mean speedup)\n")
    head = by_bins.head(20).copy()
    f.write(head.to_markdown(index=False))
    f.write("\n\n")

    f.write("## Plots\n")
    f.write("- `speedup_heatmap_*.png` (AMP/FP32 by model × workload)\n")
    f.write("- `speedup_vs_seqlen_*.png`\n")
    f.write("- `speedup_vs_bs_*.png`\n\n")

    f.write("## Methods\n")
    f.write(textwrap.dedent("""
    - We form matched pairs on (layers, d_model, bs, seqlen, compile, static).
    - Throughput (TPS) is averaged per (precision × setting) before pairing.
    - Speedup = TPS_amp / TPS_fp32. To stabilize variance, analyses use log-speedup.
    - Confidence intervals: bootstrap (B=2000) on mean log-speedup, then exponentiated.
    - Hypothesis test: exact two-sided binomial sign test on wins vs losses (ties dropped).
    - Regression: OLS of log-speedup on layers, d_model, log(bs), log(seqlen), static.
    """).strip() + "\n\n")

    f.write("## Limitations\n")
    f.write("- Synthetic tokens; real datasets may shift memory/bandwidth balance.\n")
    f.write("- Single GPU model (T4); other architectures may differ.\n")
    f.write("- No kernel-level attribution; profiler traces were disabled by default.\n\n")

    f.write("## Recommendations (T4-specific)\n")
    f.write("- Prefer AMP when small models and short sequences dominate (see top speedups).\n")
    f.write("- For high throughput targets, tune batch size first; compare AMP vs FP32 on the tuned point.\n")
    f.write("- If AMP underperforms, try enabling static padding and/or reducing sequence length.\n")

# ---------- 11) Bundle ----------
bundle = shutil.make_archive(ARTEFACTS_DIR, "zip", ARTEFACTS_DIR)

print("\n=== Research bundle ready ===")
print(f"- Artefacts dir: {ARTEFACTS_DIR}/")
print(f"- Bundle: {bundle}")
print("- Key outputs:")
print("  • full_data.csv, matched_pairs.csv, summary_by_config.csv")
print("  • stratified_bins.csv, per_precision_stats.csv")
print("  • regression_log_speedup.csv, key_metrics.json")
print("  • REPORT.md + PNG plots")

/tmp/ipython-input-2286993733.py:180: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  by_bins = (pivot.groupby(['size_bin','bs_bin','sl_bin'])



=== Research bundle ready ===
- Artefacts dir: artefacts/
- Bundle: /content/artefacts.zip
- Key outputs:
  • full_data.csv, matched_pairs.csv, summary_by_config.csv
  • stratified_bins.csv, per_precision_stats.csv
  • regression_log_speedup.csv, key_metrics.json
  • REPORT.md + PNG plots
